In [1]:
import time
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from konlpy.tag import Okt
import sys
import os
np.random.seed(1)
tf.set_random_seed(1)
# os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
okt = Okt()

In [2]:
f = open("./data/ratings_train.txt",encoding="UTF-8")
train_data = [line.split("\t")[1:] for line in f.read().splitlines()]
train_data = train_data[1:]
f = open("./data/ratings_test.txt",encoding="UTF-8")
test_data = [line.split("\t")[1:] for line in f.read().splitlines()]
test_data = test_data[1:]

In [3]:
print(len(train_data))
print(train_data[0])
print(len(test_data))
print(test_data[0])

150000
['아 더빙.. 진짜 짜증나네요 목소리', '0']
50000
['굳 ㅋ', '1']


In [4]:
for idx, data in enumerate(train_data):
    if data[0] == "":
        del train_data[idx]
for idx, data in enumerate(test_data):
    if data[0] == "":
        del test_data[idx]
print(len(train_data))
print(len(test_data))

149995
49997


In [5]:
train_tokens = [["/".join(tag) for tag in okt.pos(sentence[0], norm=True, stem=True)] for sentence in train_data]
test_tokens = [["/".join(tag) for tag in okt.pos(sentence[0], norm=True, stem=True)] for sentence in test_data]

KeyboardInterrupt: 

In [ ]:
# word2vec = Word2Vec(size=300, alpha=0.025, seed=1, sg=1, min_alpha=0.025)
# word2vec.build_vocab(tokens)
# for epoch in range(30):
#     word2vec.train(train_tokens, total_examples=word2vec.corpus_count, epochs=word2vec.iter)
#     word2vec.alpha -= 0.002
#     word2vec.min_alpha = word2vec.alpha
#     print("epochs: {}/30".format(epoch+1))
# word2vec.save("./model/word2vec.model")

In [2]:
word2vec = Word2Vec.load("./model/word2vec.model")

C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
def embedding(word):
    if word in word2vec.wv.vocab:
        return word2vec.wv[word]
    else:
        return np.random.normal(size=(300))

In [ ]:
# train_X = [[embedding(word) for word in sentence] for sentence in train_tokens]
# train_y = [[int(data[1])] for data in train_data]
# test_X = [[embedding(word) for word in sentence] for sentence in test_tokens]
# test_y = [[int(data[1])] for data in test_data]
# print(len(train_X))
# print(len(train_y))
# print(len(test_X))
# print(len(test_y))
# np.save("./data/train_X.npy",train_X)
# np.save("./data/train_y.npy",train_y)
# np.save("./data/test_X.npy",test_X)
# np.save("./data/test_y.npy",test_y)

In [ ]:
train_X = np.load("./data/train_X.npy")
train_y = np.load("./data/train_y.npy")
test_X = np.load("./data/test_X.npy")
test_y = np.load("./data/test_y.npy")
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
train_seq_length = [len(x) for x in train_X]
test_seq_length = [len(x) for x in test_X]

In [11]:
class BiLSTM:
    def __init__(self, input_size, layers, hidden_units, max_length, learning_rate, dropout_keep_prob):
        with tf.variable_scope("BiLSTM", reuse=tf.AUTO_REUSE):
            self.input_X = tf.placeholder(dtype=tf.float32, shape=[None, max_length, input_size], name="input_X")
            self.input_y = tf.placeholder(dtype=tf.float32, shape=[None,1], name="input_y")
            self.dropout_keep_prob = tf.placeholder(dtype=tf.float32, shape=(), name="dropout_keep_prob")
            self.seq_len = tf.placeholder(dtype=tf.int32, shape=[None], name="seq_len")
            
            self.output = self.bilstm(self.input_X, layers, hidden_units, self.dropout_keep_prob)
            
            self.loss = -tf.reduce_mean(self.input_y*tf.log(self.output) + (1-self.input_y)*tf.log(1-self.output))
            self.train = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
            self.pred = tf.equal(self.input_y, tf.cast(tf.to_int32(self.output >= 0.5),tf.float32))
            self.acc = tf.multiply(tf.reduce_mean(tf.cast(self.pred, tf.float32)), 100, name="acc")
            
            tf.summary.scalar(name="Loss", tensor=self.loss)
            tf.summary.scalar(name="Accuracy", tensor=self.acc)
            self.merge_graph = tf.summary.merge_all()
    
    def bilstm(self, X, layers, hidden_units, dropout_keep_prob):
        
        fw_cell_stack = []
        for i in range(layers):
            fw_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units, forget_bias=1.0, state_is_tuple=True)
            fw_cell = tf.nn.rnn_cell.DropoutWrapper(fw_cell, output_keep_prob=dropout_keep_prob)
            fw_cell_stack.append(fw_cell)
        fw_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(fw_cell_stack)
        
        bw_cell_stack = []
        for i in range(layers):
            bw_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units,forget_bias=1.0, state_is_tuple=True)
            bw_cell = tf.nn.rnn_cell.DropoutWrapper(bw_cell, output_keep_prob=dropout_keep_prob)
            bw_cell_stack.append(bw_cell)
        bw_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(bw_cell_stack)
        
#         fw_lstm_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units)
#         fw_lstm_cell = tf.nn.rnn_cell.DropoutWrapper(fw_lstm_cell, output_keep_prob=dropout_keep_prob)
#         bw_lstm_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units)
#         bw_lstm_cell = tf.nn.rnn_cell.DropoutWrapper(bw_lstm_cell, output_keep_prob=dropout_keep_prob)
        
        outputs,states = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_lstm_cell, cell_bw=bw_lstm_cell, inputs=X, dtype=tf.float32, sequence_length=self.seq_len)
        fw_output = outputs[0][:,-1]
        bw_output = outputs[1][:,0]
        
        W = tf.get_variable("W_output",shape=[2*hidden_units,1], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float32)
        b = tf.get_variable("b_output",shape=(), initializer=tf.zeros_initializer(), dtype=tf.float32)
#         output = tf.concat([states[0][1], states[1][1]], axis=1)
        output = tf.concat([fw_output, bw_output], axis=1)
        output = tf.nn.sigmoid(tf.matmul(output,W) + b, name="output")
        return output

In [18]:
input_size = 300
layers = 1
hidden_units = 128
batch_size = 32
dropout_keep_prob = 0.75
max_length = 100
learning_rate = 0.001
epochs = 2
n_class = 2

In [10]:
with tf.Graph().as_default():
    start = time.time()
    
    sess = tf.Session()
    print("Session started")
    model = BiLSTM(input_size, layers, hidden_units, max_length, learning_rate, dropout_keep_prob)
    print("Model was initialized")
    save_path = "./model/BiLSTM"
    n_batchs = (int)(len(train_X) / batch_size)
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter("./log", sess.graph)
    merge_graph = model.merge_graph
    global_steps = 0
    sess.run(tf.global_variables_initializer())
    
    def train_step(batch_X, batch_y, seq_lenght, steps, epoch):
        
        batch_X_padded = np.zeros(shape=(batch_size,max_length,input_size))
        for b in range(batch_size):
            batch_X_padded[b,:len(batch_X[b])] = batch_X[b]
        
        feed_dict = {
            model.input_X: batch_X_padded,
            model.input_y: batch_y,
            model.dropout_keep_prob: dropout_keep_prob,
            model.seq_len: seq_length
        }
        _, loss, acc = sess.run([model.train, model.loss, model.acc], feed_dict=feed_dict)
        if steps % 100 == 0:
            print("epochs: {}/{}, iters: {}, loss: {}, acc: {}".format(epoch+1,epochs,steps, loss, acc))
            summary = sess.run(merge_graph, feed_dict=feed_dict)
            writer.add_summary(summary, global_steps)
    for i in range(epochs):
        steps = 0
        for j in range(n_batchs):
            batch_X, batch_y = train_X[steps*batch_size:(steps+1)*batch_size], train_y[steps*batch_size:(steps+1)*batch_size]
            seq_length = train_seq_length[steps*batch_size:(steps+1)*batch_size]
            try:
                train_step(batch_X, batch_y, seq_length, steps, i)
            except ValueError:
                print(ValueError)
            except KeyboardInterrupt:
                saver.save(sess,save_path)
                print("Model was saved")
                sys.exit(1)
            steps += 1
            global_steps += 1
        saver.save(sess,save_path)
    saver.save(sess,save_path)
    print("Training finished")
    print("Model was saved")
    
    end = time.time()
    print("{} secs".format(end-start))

Session started


NameError: name 'BiLSTM' is not defined

In [21]:
test_len = len(test_X)

batch_size = 10000
batchs = int(test_len/batch_size)

with tf.Graph().as_default():
    sess = tf.Session()
    print("Session started")
    save_path = "./model/BiLSTM"
    saver = tf.train.import_meta_graph(save_path+".meta")
    saver.restore(sess, save_path)
    graph = tf.get_default_graph()
    print("Model was loaded")
    
    input_X = graph.get_tensor_by_name("BiLSTM/input_X:0")
    input_y = graph.get_tensor_by_name("BiLSTM/input_y:0")
    dropout_keep_prob = graph.get_tensor_by_name("BiLSTM/dropout_keep_prob:0")
    output = graph.get_tensor_by_name("BiLSTM/output:0")
    seq_len = graph.get_tensor_by_name("BiLSTM/seq_len:0")
    acc = graph.get_tensor_by_name("BiLSTM/acc:0")
    
    steps = 0
    acc_total = 0
    for b in range(batchs):
        batch_X, batch_y = test_X[steps*batch_size:(steps+1)*batch_size], test_y[steps*batch_size:(steps+1)*batch_size]
        batch_X_padded = np.zeros(shape=(batch_size,max_length,input_size))
        for b in range(batch_size):
            batch_X_padded[b,:len(batch_X[b])] = batch_X[b]
        seq_len_ = test_seq_length[steps*batch_size:(steps+1)*batch_size]
        feed_dict = {
            input_X: batch_X_padded,
            input_y: batch_y,
            dropout_keep_prob: 1.0,
            seq_len: seq_len_
        }
        acc_batch = sess.run(acc, feed_dict=feed_dict)
        acc_total += acc_batch
        steps += 1
        print("Batch {} 정확도: {}".format(steps,acc_batch))
    batch_X, batch_y = test_X[steps*batch_size:], test_y[steps*batch_size:]
    batch_size = len(batch_X)
    print("남은 데이터 수: {}".format(batch_size))
    batch_X_padded = np.zeros(shape=(batch_size,max_length,input_size))
    for b in range(batch_size):
        batch_X_padded[b,:len(batch_X[b])] = batch_X[b]
    seq_len_ = test_seq_length[steps*batch_size:(steps+1)*batch_size]
    feed_dict = {
        input_X: batch_X_padded,
        input_y: batch_y,
        dropout_keep_prob: 1.0,
        seq_len: seq_len_
    }
    acc_batch = sess.run(acc, feed_dict=feed_dict)
    acc_total += acc_batch
    steps += 1
    print("Batch {} 정확도: {}".format(steps,acc_batch))
    print("전체 정확도: {}".format(acc_total/(steps)))

NameError: name 'test_X' is not defined

In [25]:
batch_size = 5

with tf.Graph().as_default():
    sess = tf.Session()
    save_path = "./model/BiLSTM"
    saver = tf.train.import_meta_graph(save_path+".meta")
    saver.restore(sess, save_path)
    graph = tf.get_default_graph()
    
    input_X = graph.get_tensor_by_name("BiLSTM/input_X:0")
    input_y = graph.get_tensor_by_name("BiLSTM/input_y:0")
    dropout_keep_prob = graph.get_tensor_by_name("BiLSTM/dropout_keep_prob:0")
    output = graph.get_tensor_by_name("BiLSTM/output:0")
    seq_len = graph.get_tensor_by_name("BiLSTM/seq_len:0")
    acc = graph.get_tensor_by_name("BiLSTM/acc:0")
    
    X = [
        "우와 영화 진짜 재밌다.",
        "연기 개못하네 ㅡㅡ",
        "명품 연기.. 역시",
        "돈 아깝다",
        "이런걸 영화라고 만듬..?"
    ]
    token_X = [["/".join(tag) for tag in okt.pos(sentence, norm=True, stem=True)] for sentence in X]
    batch_X = [[embedding(word) for word in sentence] for sentence in token_X]
    batch_X_padded = np.zeros(shape=(batch_size,max_length,input_size))
    for b in range(batch_size):
        batch_X_padded[b,:len(batch_X[b])] = batch_X[b]
    seq_len_ = [len(x) for x in batch_X]
    feed_dict = {
        input_X: batch_X_padded,
        dropout_keep_prob: 1.0,
        seq_len: seq_len_
    }
    pred = sess.run(output, feed_dict=feed_dict)
    for idx, p in enumerate(pred):
        print("리뷰 \"{}\" 은 약 {}%의 확률로 긍정 리뷰입니다.".format(X[idx], int(p[0]*100)))
    batch_size = 1
    while True:
        X = input("리뷰를 입력하세요.(exit => 종료)")
        if X == "exit":
            break
        X = [X]
        token_X = [["/".join(tag) for tag in okt.pos(sentence, norm=True, stem=True)] for sentence in X]
        batch_X = [[embedding(word) for word in sentence] for sentence in token_X]
        batch_X_padded = np.zeros(shape=(batch_size, max_length, input_size))
        for b in range(batch_size):
            batch_X_padded[b, :len(batch_X[b])] = batch_X[b]
        seq_len_ = [len(x) for x in X]
        feed_dict = {
            input_X: batch_X_padded,
            dropout_keep_prob: 0.0,
            seq_len: seq_len_
        }
        pred = sess.run(output, feed_dict=feed_dict)
        print("리뷰 \"{}\" 은 약 {}%의 확률로 긍정 리뷰입니다.".format(X[0], int(pred[0]*100)))

리뷰 "우와 영화 진짜 재밌다." 은 약 98%의 확률로 긍정 리뷰입니다.
리뷰 "연기 개못하네 ㅡㅡ" 은 약 4%의 확률로 긍정 리뷰입니다.
리뷰 "명품 연기.. 역시" 은 약 96%의 확률로 긍정 리뷰입니다.
리뷰 "돈 아깝다" 은 약 0%의 확률로 긍정 리뷰입니다.
리뷰 "이런걸 영화라고 만듬..?" 은 약 8%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)영화 대박이네
리뷰 "영화 대박이네" 은 약 83%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)영화 대박이네
리뷰 "영화 대박이네" 은 약 82%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)영화 대박이네
리뷰 "영화 대박이네" 은 약 82%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)영화 대박이네
리뷰 "영화 대박이네" 은 약 80%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)영화 대박이네
리뷰 "영화 대박이네" 은 약 82%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)exit


In [3]:
# savedmodel로 저장
# tensorflow-serving에서 사용할 모델 저장할 경로
export_path = "./model/BiLSTM/1"
with tf.Graph().as_default():
    sess = tf.Session()
#     tf.Saver로 저장한 모델을 불러올 경로
    save_path = "./model/BiLSTM"
    saver = tf.train.import_meta_graph(save_path+".meta")
    saver.restore(sess, save_path)
    graph = tf.get_default_graph()
    input_X = graph.get_tensor_by_name("BiLSTM/input_X:0")
    dropout_keep_prob = graph.get_tensor_by_name("BiLSTM/dropout_keep_prob:0")
    seq_len = graph.get_tensor_by_name("BiLSTM/seq_len:0")
    output = graph.get_tensor_by_name("BiLSTM/output:0")
#     tensorflow-serving에서 사용할 모델 형식으로 저장
    tf.saved_model.simple_save(sess, export_path, 
                               inputs={
                                   "input_X": input_X,
                                   "dropout_keep_prob": dropout_keep_prob,
                                   "seq_len": seq_len
                               }, 
                               outputs={
                                   "output": output
                               })
    